In [33]:
#Import libraries
import numpy as np
import scipy as sp

import plotly.plotly as py
py.sign_in('sarah_c', 'NB0yoorG3OmL0rdfPd6h')
from plotly.tools import FigureFactory as ff
import plotly.graph_objs as go
import pandas as pd

import datetime
import urllib
 
from bokeh.plotting import *
from bokeh.models import HoverTool
from collections import OrderedDict

from sklearn.linear_model import LinearRegression
import statsmodels.formula.api as smf

In [5]:
#2016
# Grab contribution data
url_2016 = 'https://data.wa.gov/resource/74eq-kst5.json?$limit=450000&$offset=1000&\
$where=receipt_date%20between%20%272016-01-01T00:00:00%27%20and%20%272016-12-31T00:00:00%27&\
$order=receipt_date'

#Read in data
raw_data_2016 = pd.read_json(url_2016)

In [29]:
#check 2016 contribution data
#raw_data_2016.tail(10)
raw_data_2016.shape

raw_data_2016['receipt_date'] = pd.to_datetime(raw_data_2016['receipt_date'])

In [30]:
#GOP
gop = raw_data_2016[raw_data_2016.party=='REPUBLICAN']

#dems
dems = raw_data_2016[raw_data_2016.party=='DEMOCRAT']

#all the rest
all_other = raw_data_2016[raw_data_2016.party!='REPUBLICAN']
all_other = all_other[all_other.party!='DEMOCRAT']

#view
gop.head(5)
dems.head(5)
all_other.head(5)

,amount,ballot_number,cash_or_in_kind,code,contributor_address,contributor_city,contributor_employer_city,contributor_employer_name,contributor_employer_state,contributor_location,...,office,origin,party,position,primary_general,receipt_date,report_number,type,url,url_description
0,3.0,NaN,Cash,Individual,"1 JEFFERSON PARKWAY, APT. 14",LAKE OSWEGO,NaN,NaN,NaN,"{'coordinates': [-122.7157, 45.43227], 'type':...",...,NaN,C3,NaN,NaN,Full election cycle,2016-01-04,100673897,Political Committee,http://web.pdc.wa.gov/rptimg/default.aspx?batc...,View report
1,3.5,NaN,Cash,Individual,4202 FEDERAL,EVERETT,NaN,NaN,NaN,"{'coordinates': [-122.21797, 47.96238], 'type'...",...,NaN,C3,NaN,NaN,Full election cycle,2016-01-04,100673897,Political Committee,http://web.pdc.wa.gov/rptimg/default.aspx?batc...,View report
2,3.0,NaN,Cash,Individual,1809 BROADWAY EAST,SEATTLE,NaN,NaN,NaN,"{'coordinates': [-122.32134, 47.63532], 'type'...",...,NaN,C3,NaN,NaN,Full election cycle,2016-01-04,100673897,Political Committee,http://web.pdc.wa.gov/rptimg/default.aspx?batc...,View report
3,2.5,NaN,Cash,Individual,24427 B RICHARDS ROAD,SEDRO WOOLLEY,NaN,NaN,NaN,"{'coordinates': [-122.21628, 48.51088], 'type'...",...,NaN,C3,NaN,NaN,Full election cycle,2016-01-04,100673897,Political Committee,http://web.pdc.wa.gov/rptimg/default.aspx?batc...,View report
4,1.4,NaN,Cash,Individual,1932 NORTH 120TH,SEATTLE,NaN,NaN,NaN,"{'coordinates': [-122.33438, 47.71611], 'type'...",...,NaN,C3,NaN,NaN,Full election cycle,2016-01-04,100673897,Political Committee,http://web.pdc.wa.gov/rptimg/default.aspx?batc...,View report


In [31]:
#Group by receipt date
grouped_dems = dems.groupby(by=['receipt_date']).sum()
grouped_gop = gop.groupby(by=['receipt_date']).sum()
grouped_other = all_other.groupby(by=['receipt_date']).sum()

grouped_dems.shape

(360, 5)

In [8]:
#Load National 2016 set
nats16 = pd.read_table('16-US-Pres-GE TrumpvClinton-poll-responses-clean.tsv')

#View data
nats16.head(10)

,Trump,Clinton,Johnson,Other,Undecided,poll_slug,survey_house,start_date,end_date,question_text,sample_subpopulation,observations,margin_of_error,mode,partisanship,partisan_affiliation
0,41.0,45.0,4.0,2.0,8.0,insights-west-26812,Insights West,2016-11-04,2016-11-07,"As you may know, there will be a presidential ...",Likely Voters,940.0,3.2,Internet,Nonpartisan,None
1,6.0,89.0,1.0,0.0,4.0,insights-west-26812,Insights West,2016-11-04,2016-11-07,"As you may know, there will be a presidential ...",Likely Voters - Democrat,NaN,NaN,Internet,Nonpartisan,None
2,82.0,7.0,3.0,2.0,6.0,insights-west-26812,Insights West,2016-11-04,2016-11-07,"As you may know, there will be a presidential ...",Likely Voters - Republican,NaN,NaN,Internet,Nonpartisan,None
3,38.0,43.0,8.0,4.0,7.0,insights-west-26812,Insights West,2016-11-04,2016-11-07,"As you may know, there will be a presidential ...",Likely Voters - independent,NaN,NaN,Internet,Nonpartisan,None
4,43.0,41.0,7.0,4.0,5.0,ibd-tipp-26811,IBD/TIPP,2016-11-04,2016-11-07,NaN,Likely Voters,1107.0,3.1,Live Phone,Nonpartisan,None
5,42.0,43.0,NaN,5.0,9.0,ibd-tipp-26811,IBD/TIPP,2016-11-04,2016-11-07,NaN,Likely Voters,1107.0,3.1,Live Phone,Nonpartisan,None
6,41.0,45.0,5.0,5.0,4.0,yougov-economist-26805,YouGov/Economist,2016-11-04,2016-11-07,NaN,Likely Voters,3669.0,NaN,Internet,Nonpartisan,None
7,6.0,90.0,1.0,2.0,2.0,yougov-economist-26805,YouGov/Economist,2016-11-04,2016-11-07,NaN,Likely Voters - Democrat,1392.0,NaN,Internet,Nonpartisan,None
8,84.0,4.0,4.0,4.0,4.0,yougov-economist-26805,YouGov/Economist,2016-11-04,2016-11-07,NaN,Likely Voters - Republican,1110.0,NaN,Internet,Nonpartisan,None
9,44.0,31.0,9.0,9.0,8.0,yougov-economist-26805,YouGov/Economist,2016-11-04,2016-11-07,NaN,Likely Voters - independent,1167.0,NaN,Internet,Nonpartisan,None


In [21]:
#Group nats16 by end_date & find average
nats = nats16.groupby(by=['end_date']).mean()

#view nats
nats.head(25)
nats.shape

(259, 7)

In [28]:
nats.head(10)

,Trump,Clinton,Johnson,Other,Undecided,observations,margin_of_error,key
end_date,,,,,,,,
2015-05-26,34.250000,48.750000,NaN,2.50,14.000000,1046.000000,3.266667,2015-05-26
2015-06-22,35.000000,47.250000,NaN,NaN,17.750000,420.750000,NaN,2015-06-22
2015-06-23,34.000000,51.000000,NaN,3.00,12.000000,1005.000000,3.000000,2015-06-23
2015-06-28,36.750000,57.000000,NaN,6.00,0.000000,890.000000,3.500000,2015-06-28
2015-07-12,35.250000,49.500000,NaN,NaN,16.000000,499.250000,3.000000,2015-07-12
2015-07-21,38.750000,47.250000,NaN,NaN,13.750000,702.333333,3.900000,2015-07-21
2015-07-25,40.500000,55.750000,NaN,3.75,0.000000,898.000000,3.500000,2015-07-25
2015-07-28,38.777778,50.444444,NaN,3.50,9.333333,989.800000,3.300000,2015-07-28
2015-08-01,49.000000,50.000000,NaN,NaN,NaN,3477.000000,2.000000,2015-08-01


In [66]:
#create  dems key
grouped_dems['key'] = grouped_dems.index.astype(str)
nats['key'] = nats.index.astype(str)
#merge df's
merged_dems = pd.merge(grouped_dems, nats, how='outer', on=['key'])

#change to dataframe
merged_dems = pd.DataFrame(merged_dems)


#create gop key
grouped_gop['key'] = grouped_gop.index.astype(str)

#merge df's
merged_gop = pd.merge(grouped_gop, nats, how='outer', on=['key'])

#Change to dataframe
merged_gop = pd.DataFrame(merged_gop)

#create other key
grouped_other['key'] = grouped_other.index.astype(str)

#merge df's
merged_other = pd.merge(grouped_other, nats, how='outer', on=['key'])

#Change to dataframe
merged_other = pd.DataFrame(merged_other)



In [51]:
#Dems Linear Regression with Clinton
dems_ols = smf.ols('amount ~ Clinton', data = merged_dems).fit()
print("DEMS CLINTON:")
print(dems_ols.summary())

# Repubs LR with Clinton
gop_ols = smf.ols('amount ~ Clinton', data = merged_gop).fit()
print("GOP CLINTON:")
print(gop_ols.summary())

#Other LR with Clinton
other_ols = smf.ols('amount ~ Clinton', data = merged_other).fit()
print('OTHER CLINTON:')
print(other_ols.summary())



DEMS CLINTON:
                            OLS Regression Results                            
Dep. Variable:                 amount   R-squared:                       0.024
Model:                            OLS   Adj. R-squared:                  0.019
Method:                 Least Squares   F-statistic:                     4.839
Date:                Tue, 16 May 2017   Prob (F-statistic):             0.0290
Time:                        08:49:49   Log-Likelihood:                -2557.7
No. Observations:                 200   AIC:                             5119.
Df Residuals:                     198   BIC:                             5126.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept   2.689e+05   8.34e+04      

In [63]:
# Split dems into train and test
#generate indexes
indexes = np.random.choice(419,int(merged_dems.shape[0]*0.70),replace=False)

#create train with the random sample graph
dems_train = merged_dems.ix[indexes,].reset_index()

#select rows not in index
dems_test = merged_dems.ix[~merged_dems.index.isin(indexes),].reset_index()

#Verify the split is 70/30
print(dems_train.shape, dems_test.shape)


(293, 14) (126, 14)


In [64]:
# Train Dems Linear Regression with Clinton
dems_model_1 = smf.ols('amount ~ Clinton', data = dems_train).fit()
print("DEMS CLINTON:")
print(dems_model_1.summary())


DEMS CLINTON:
                            OLS Regression Results                            
Dep. Variable:                 amount   R-squared:                       0.014
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     1.873
Date:                Tue, 16 May 2017   Prob (F-statistic):              0.173
Time:                        13:19:11   Log-Likelihood:                -1767.9
No. Observations:                 138   AIC:                             3540.
Df Residuals:                     136   BIC:                             3546.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept   2.206e+05   9.73e+04      

In [ ]:
#See how dems_model_1 performs on test set


In [ ]:
#Calculate RMSE for Each Linear Regression Model 
#Define RMSE Function
def compute_rmse(predictions, yvalues):
    rmse = np.sqrt(((predictions - yvalues) ** 2).mean()) # Add RMSE function
    return rmse


In [ ]:
#Calculate RMSE

In [67]:
# Split GOP into train and test
#generate indexes
indexes_gop = np.random.choice(419,int(merged_gop.shape[0]*0.70),replace=False)

#create train with the random sample graph
gop_train = merged_gop.ix[indexes_gop,].reset_index()

#select rows not in index
gop_test = merged_gop.ix[~merged_gop.index.isin(indexes_gop),].reset_index()

#Verify the split is 70/30
print(gop_train.shape, gop_test.shape)

(291, 14) (128, 14)


In [68]:
# Train GOP Linear Regression with Clinton
gop_model_1 = smf.ols('amount ~ Clinton', data = gop_train).fit()
print("GOP CLINTON:")
print(gop_model_1.summary())

GOP CLINTON:
                            OLS Regression Results                            
Dep. Variable:                 amount   R-squared:                       0.026
Model:                            OLS   Adj. R-squared:                  0.019
Method:                 Least Squares   F-statistic:                     3.625
Date:                Wed, 17 May 2017   Prob (F-statistic):             0.0590
Time:                        19:48:44   Log-Likelihood:                -1792.1
No. Observations:                 139   AIC:                             3588.
Df Residuals:                     137   BIC:                             3594.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept   2.736e+05   1.06e+05      2

In [ ]:
# See how well GOP model performs on test set

In [ ]:
# Calculate GOP RMSE

In [69]:
# Split other into train and test
#generate indexes
indexes_other = np.random.choice(419,int(merged_other.shape[0]*0.70),replace=False)

#create train with the random sample graph
other_train = merged_other.ix[indexes_other,].reset_index()

#select rows not in index
other_test = merged_other.ix[~merged_other.index.isin(indexes_other),].reset_index()

#Verify the split is 70/30
print(other_train.shape, other_test.shape)

(295, 14) (127, 14)


In [70]:
# Train Other Linear Regression with Clinton
other_model_1 = smf.ols('amount ~ Clinton', data = other_train).fit()
print("OTHER CLINTON:")
print(other_model_1.summary())

OTHER CLINTON:
                            OLS Regression Results                            
Dep. Variable:                 amount   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                 -0.005
Method:                 Least Squares   F-statistic:                    0.2852
Date:                Wed, 17 May 2017   Prob (F-statistic):              0.594
Time:                        19:53:46   Log-Likelihood:                -1975.6
No. Observations:                 133   AIC:                             3955.
Df Residuals:                     131   BIC:                             3961.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept   8.528e+05   8.05e+05     

In [ ]:
# See how well Other model performs on test set

In [ ]:
# Calculate Other RMSE

In [42]:
#Dems LR with Trump
dems_ols_trump = smf.ols('amount ~ Trump', data = merged_dems).fit()
print('DEMS TRUMP:')
print(dems_ols_trump.summary())

#Repubs LR with Trump
gop_ols_trump = smf.ols('amount ~ Trump', data = merged_gop).fit()
print('GOP TRUMP:')
print(gop_ols_trump.summary())

#Other LR with Trump
other_ols_trump = smf.ols('amount ~ Trump', data = merged_other).fit()
print('OTHER TRUMP:')
print(other_ols_trump.summary())

DEMS TRUMP:
                            OLS Regression Results                            
Dep. Variable:                 amount   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.005
Method:                 Least Squares   F-statistic:                   0.06411
Date:                Sun, 14 May 2017   Prob (F-statistic):              0.800
Time:                        19:35:31   Log-Likelihood:                -2560.0
No. Observations:                 200   AIC:                             5124.
Df Residuals:                     198   BIC:                             5131.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept   1.068e+05   8.29e+04      1.

In [9]:
#Washington state Specific presidential polls
wa_pres = pd.read_table('16-WA-Pres-GE TrumpvClinton-poll-responses-clean.tsv')

#View data
wa_pres.head(10)

#http://elections.huffingtonpost.com/pollster/2016-washington-president-trump-vs-clinton

,Trump,Clinton,Other,Undecided,poll_slug,survey_house,start_date,end_date,question_text,sample_subpopulation,observations,margin_of_error,mode,partisanship,partisan_affiliation
0,33.0,51.0,13.0,3.0,surveymonkey-26807,SurveyMonkey,2016-11-01,2016-11-07,NaN,Likely Voters,1451,NaN,Internet,Nonpartisan,None
1,36.0,51.0,6.0,7.0,insights-west-26816,Insights West,2016-11-04,2016-11-06,"As you may know, there will be a presidential ...",Likely Voters,402,4.9,Internet,Nonpartisan,None
2,39.0,56.0,6.0,NaN,upi-cvoter-26793,UPI/CVOTER,2016-10-30,2016-11-06,NaN,Likely Voters,328,NaN,Internet,Nonpartisan,None
3,34.0,51.0,12.0,3.0,surveymonkey-26769,SurveyMonkey,2016-10-30,2016-11-06,NaN,Likely Voters,1292,NaN,Internet,Nonpartisan,None
4,38.0,47.0,NaN,15.0,ipsos-reuters-26806,Ipsos/Reuters,2016-10-17,2016-11-06,NaN,Likely Voters,863,NaN,Internet,Nonpartisan,None
5,34.0,52.0,12.0,2.0,surveymonkey-26731,SurveyMonkey,2016-10-29,2016-11-04,NaN,Likely Voters,1205,NaN,Internet,Nonpartisan,None
6,34.0,53.0,12.0,2.0,surveymonkey-26694,SurveyMonkey,2016-10-28,2016-11-03,NaN,Likely Voters,944,NaN,Internet,Nonpartisan,None
7,39.0,46.0,NaN,15.0,ipsos-reuters-26743,Ipsos/Reuters,2016-10-14,2016-11-03,NaN,Likely Voters,870,NaN,Internet,Nonpartisan,None
8,38.0,50.0,9.0,4.0,surveyusa-26738,SurveyUSA,2016-10-31,2016-11-02,NaN,Likely Voters,681,3.8,IVR/Online,Nonpartisan,None
9,34.0,54.0,12.0,1.0,surveymonkey-26660,SurveyMonkey,2016-10-27,2016-11-02,NaN,Likely Voters,807,NaN,Internet,Nonpartisan,None


In [11]:
#Washington Gov. Bryant vs. inslee
wa_govs = pd.read_table('16-WA-Gov-GE-BvI-poll-responses-clean.tsv')

wa_govs.head(10)
#http://elections.huffingtonpost.com/pollster/2016-washington-governor-bryant-vs-inslee

,Bryant,Inslee,Other,Undecided,poll_slug,survey_house,start_date,end_date,question_text,sample_subpopulation,observations,margin_of_error,mode,partisanship,partisan_affiliation
0,42.0,55.0,NaN,3.0,surveymonkey-26809,SurveyMonkey,2016-11-01,2016-11-07,NaN,Likely Voters,1451,NaN,Internet,Nonpartisan,None
1,40.0,49.0,NaN,10.0,insights-west-26816,Insights West,2016-11-04,2016-11-06,"Also on Nov. 8, there will be an election for ...",Likely Voters,402,4.9,Internet,Nonpartisan,None
2,42.0,55.0,NaN,3.0,surveymonkey-26782,SurveyMonkey,2016-10-31,2016-11-06,NaN,Likely Voters,1292,NaN,Internet,Nonpartisan,None
3,41.0,56.0,NaN,3.0,surveymonkey-26713,SurveyMonkey,2016-10-28,2016-11-03,NaN,Likely Voters,944,NaN,Internet,Nonpartisan,None
4,43.0,50.0,NaN,6.0,surveyusa-26738,SurveyUSA,2016-10-31,2016-11-02,NaN,Likely Voters,667,3.9,IVR/Online,Nonpartisan,None
5,41.0,56.0,NaN,3.0,surveymonkey-26687,SurveyMonkey,2016-10-27,2016-11-02,NaN,Likely Voters,807,NaN,Internet,Nonpartisan,None
6,41.0,55.0,NaN,4.0,surveymonkey-26635,SurveyMonkey,2016-10-26,2016-11-01,NaN,Likely Voters,698,NaN,Internet,Nonpartisan,None
7,42.0,55.0,NaN,3.0,surveymonkey-26633,SurveyMonkey,2016-10-25,2016-10-31,NaN,Likely Voters,745,NaN,Internet,Nonpartisan,None
8,45.0,51.0,NaN,13.0,kcts-9-crosscut-washington-26512,KCTS 9/Crosscut/Washington,2016-10-06,2016-10-13,NaN,Likely Voters,750,NaN,Internet,Nonpartisan,None
9,40.0,50.0,2.0,8.0,strategies-360-d-komo-news-25991,Strategies 360 (D- KOMO News),2016-09-29,2016-10-03,NaN,Likely Voters,500,4.4,Live Phone,Sponsor,Dem


In [12]:
#Washington state senate - vance vs. murray
wa_sen = pd.read_table('16-WA-Sen-GE-VvM-poll-responses-clean.tsv')

wa_sen.head(10)
#http://elections.huffingtonpost.com/pollster/2016-washington-senate-vance-vs-murray

,Vance,Murray,Other,Undecided,poll_slug,survey_house,start_date,end_date,question_text,sample_subpopulation,observations,margin_of_error,mode,partisanship,partisan_affiliation
0,37.0,59.0,NaN,4.0,surveymonkey-26808,SurveyMonkey,2016-11-01,2016-11-07,NaN,Likely Voters,1451,NaN,Internet,Nonpartisan,None
1,37.0,53.0,NaN,11.0,insights-west-26816,Insights West,2016-11-04,2016-11-06,"Also on Nov. 8, there will be an election for ...",Likely Voters,402,4.9,Internet,Nonpartisan,None
2,37.0,59.0,NaN,4.0,surveymonkey-26767,SurveyMonkey,2016-10-31,2016-11-06,NaN,Likely Voters,1292,NaN,Internet,Nonpartisan,None
3,36.0,61.0,NaN,3.0,surveymonkey-26712,SurveyMonkey,2016-10-28,2016-11-03,NaN,Likely Voters,944,NaN,Internet,Nonpartisan,None
4,41.0,53.0,NaN,6.0,surveyusa-26738,SurveyUSA,2016-10-31,2016-11-02,NaN,Likely Voters,667,3.9,IVR/Online,Nonpartisan,None
5,36.0,61.0,NaN,3.0,surveymonkey-26686,SurveyMonkey,2016-10-27,2016-11-02,NaN,Likely Voters,807,NaN,Internet,Nonpartisan,None
6,36.0,61.0,NaN,3.0,surveymonkey-26631,SurveyMonkey,2016-10-26,2016-11-01,NaN,Likely Voters,698,NaN,Internet,Nonpartisan,None
7,36.0,61.0,NaN,3.0,surveymonkey-26610,SurveyMonkey,2016-10-25,2016-10-31,NaN,Likely Voters,745,NaN,Internet,Nonpartisan,None
8,39.0,55.0,NaN,18.0,kcts-9-crosscut-washington-26512,KCTS 9/Crosscut/Washington,2016-10-06,2016-10-13,NaN,Likely Voters,750,NaN,Internet,Nonpartisan,None
9,25.0,46.0,2.0,27.0,insights-west-25627,Insights West,2016-09-12,2016-09-14,"Also on Nov. 8, there will be an election for ...",Likely Voters,505,4.4,Internet,Nonpartisan,None
